In [1]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

def clean_column_names(df):
    columns = []
    for each in df.columns:
        if "+/-" in each:
            each = each.replace("+/-", "_plus_minus")
        if "#" in each:
            each = each.replace("#", "num")
        if "/" in each:
            each = each.replace("/", "_per_")
        if "%" in each:
            each = each.replace("%", "_pct")
        if "-" in each:
            each = each.replace("-", "_")
        if ":" in each:
            each = each.replace(":", "_")
        if "+" in each:
            each = each.replace("+", "_and_")
        
        each = each.lower()
        
        columns.append(each)
        
    df.columns = columns

    df["squad"] = df["squad"].str.lower()
    
    return df
    

def flatten_df(df):
    try:
        #Flatten the dataframe by reducing the levels of the columns
        df.columns = ["_".join([each.strip().replace(" ", "") for each in i]) if "Unnamed" not in i[0] else i[-1].strip().replace(" ", "_") for i in df.columns]
    except Exception as e:
        #Error in Flattening the dataframe
        print("Error occurred unable to flatten the dataframe.")
        print(f"Error message: {str(e)}")
    finally:
        #Return the DataFrame
        return df

def transform_combine(raw_squad_df, raw_opponent_df):
    #Squad Dataframe: Flatten and Creating the Value colum
    raw_squad_df = flatten_df(raw_squad_df.copy())
    raw_squad_df.loc[:, "Value"] = "squad"
    
    #Opponent Dataframe: Flatten, removing the string "VS", and Creating the Value colum
    raw_opponent_df = flatten_df(raw_opponent_df.copy())
    raw_opponent_df.loc[:, "Squad"] = raw_opponent_df.loc[:, "Squad"].apply(lambda x: " ".join(x.split()[1:]))
    raw_opponent_df.loc[:, "Value"] = "opponent"
    
    try:
        #Appending the dataframes
        stats_df = raw_opponent_df.append(raw_squad_df, ignore_index=True)
    
    except Exception as e:
        #Error in Flattening the dataframe
        print("Error occurred unable to append the dataframes.")
        print(f"Error message: {str(e)}")
    
    finally:
        #Return the Appended DataFrame
        return stats_df

def pushToDB(table_name, df, conn, if_exists = "replace", index = False):
    
    df = clean_column_names(df.copy())
    
    # Begin a transaction
    transaction = conn.begin()
    
    try:
        # Push the DataFrame to PostgreSQL
        df.to_sql(name= table_name, con= conn, if_exists= if_exists, index= index)
        
        # Commit the transaction
        transaction.commit()
        print(f"{table_name} has been successfully committed.")
        
    except Exception as e:
        # Rollback the transaction if there's an error
        transaction.rollback()
        
        print(f"Error occurred in uploading {table_name}. Transaction has been rolled back.")
        print(f"Error message: {str(e)}")

#Link to Premier League
PL_STATS_URL = r"https://fbref.com/en/comps/9/Premier-League-Stats"
PL_SCORES_FIXTURES_URL = r"https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"

print("Data Extract Phase Started....")

raw_scores_and_fixtures = pd.read_html(PL_SCORES_FIXTURES_URL)[0]

all_tables = pd.read_html(PL_STATS_URL)

raw_regular_season_overall = all_tables[0]
raw_regular_season_home_away = all_tables[1]

raw_squad_standard_stats_squad = all_tables[2]
raw_squad_standard_stats_opponent = all_tables[3]

raw_squad_goalkeeping_squad = all_tables[4]
raw_squad_goalkeeping_opponent = all_tables[5]

raw_squad_advanced_goalkeeping_squad = all_tables[6]
raw_squad_advanced_goalkeeping_opponent = all_tables[7]

raw_squad_shooting_squad = all_tables[8]
raw_squad_shooting_opponent = all_tables[9]

raw_squad_passing_squad = all_tables[10]
raw_squad_passing_opponent = all_tables[11]

raw_squad_pass_types_squad = all_tables[12]
raw_squad_pass_types_opponent = all_tables[13]

raw_squad_goal_shot_creation_squad = all_tables[14]
raw_squad_goal_shot_creation_opponent = all_tables[15]

raw_squad_defensive_actions_squad = all_tables[16]
raw_squad_defensive_actions_opponent = all_tables[17]

raw_squad_possession_squad = all_tables[18]
raw_squad_possession_opponent = all_tables[19]

raw_squad_playing_time_squad = all_tables[20]
raw_squad_playing_time_opponent = all_tables[21]

raw_squad_miscellaneous_stats_squad = all_tables[22]
raw_squad_miscellaneous_stats_opponent = all_tables[23]

print("Data Extract Phase Ended....")

print("Data Transformation Phase Started....")

print("Regular Season Transformations....")
raw_regular_season_home_away = flatten_df(raw_regular_season_home_away.copy())
raw_regular_season_overall.columns = ["Overall_"+i.strip().replace(" ", "") if i not in ["Rk", "Squad"] else i for i in raw_regular_season_overall.columns]
regular_season = raw_regular_season_overall.merge(right=raw_regular_season_home_away, how='inner', on=["Rk", "Squad"], validate="one_to_one")

print("Standard Stats Transformations....")
standard_stats = transform_combine(raw_squad_df= raw_squad_standard_stats_squad.copy(), raw_opponent_df= raw_squad_standard_stats_opponent.copy())

print("Goalkeeping Stats Transformations....")
goalkeeping_stats = transform_combine(raw_squad_df= raw_squad_goalkeeping_squad.copy(), raw_opponent_df= raw_squad_goalkeeping_opponent.copy())

print("Advanced Goalkeeping Stats Transformations....")
advanced_goalkeeping_stats = transform_combine(raw_squad_df= raw_squad_advanced_goalkeeping_squad.copy(), raw_opponent_df= raw_squad_advanced_goalkeeping_opponent.copy())

print("Shooting Stats Transformations....")
shooting_stats = transform_combine(raw_squad_df= raw_squad_shooting_squad.copy(), raw_opponent_df= raw_squad_shooting_opponent.copy())

print("Passing Stats Transformations....")
passing_stats = transform_combine(raw_squad_df= raw_squad_passing_squad.copy(), raw_opponent_df= raw_squad_passing_opponent.copy())

print("Passing Types Stats Transformations....")
passing_types_stats = transform_combine(raw_squad_df= raw_squad_pass_types_squad.copy(), raw_opponent_df= raw_squad_pass_types_opponent.copy())

print("Goal Shot Creation Stats Transformations....")
goal_shot_creation_stats = transform_combine(raw_squad_df= raw_squad_goal_shot_creation_squad.copy(), raw_opponent_df= raw_squad_goal_shot_creation_opponent.copy())

print("Defensive Action Stats Transformations....")
defensive_action_stats = transform_combine(raw_squad_df= raw_squad_defensive_actions_squad.copy(), raw_opponent_df= raw_squad_defensive_actions_opponent.copy())

print("Posession Stats Transformations....")
possession_stats = transform_combine(raw_squad_df= raw_squad_possession_squad.copy(), raw_opponent_df= raw_squad_possession_opponent.copy())

print("Playing Time Stats Transformations....")
playing_time_stats = transform_combine(raw_squad_df= raw_squad_playing_time_squad.copy(), raw_opponent_df= raw_squad_playing_time_opponent.copy())

print("Miscellaneous Stats Transformations....")
miscellaneous_stats = transform_combine(raw_squad_df= raw_squad_miscellaneous_stats_squad.copy(), raw_opponent_df= raw_squad_miscellaneous_stats_opponent.copy())

print("Data Transformation Phase Ended....")

database="football-db"
user='user'
password='password'
host='192.168.59.101'
port= '30432'

try:
    print("Establishing Connection with DB....")
    db = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
    conn = db.connect()
    print("Successfully Established Connection with DB....")
    
except Exception as e:
    print("Unable to Establish Connection with DB....")
    raise(e)


print("Data Loading Phase Started....")

#Creating/Updating the regular_season database
pushToDB(table_name="regular_season", df=regular_season, conn=conn)

#Creating/Updating the standard_stats database
pushToDB(table_name="standard_stats", df=standard_stats, conn=conn)

#Creating/Updating the goalkeeping_stats database
pushToDB(table_name="goalkeeping_stats", df=goalkeeping_stats, conn=conn)

#Creating/Updating the advanced_goalkeeping_stats database
pushToDB(table_name="advanced_goalkeeping_stats", df=advanced_goalkeeping_stats, conn=conn)

#Creating/Updating the shooting_stats database
pushToDB(table_name="shooting_stats", df=shooting_stats, conn=conn)

#Creating/Updating the passing_stats database
pushToDB(table_name="passing_stats", df=passing_stats, conn=conn)

#Creating/Updating the passing_types_stats database
pushToDB(table_name="passing_types_stats", df=passing_types_stats, conn=conn)

#Creating/Updating the goal_shot_creation_stats database
pushToDB(table_name="goal_shot_creation_stats", df=goal_shot_creation_stats, conn=conn)

#Creating/Updating the defensive_action_stats database
pushToDB(table_name="defensive_action_stats", df=defensive_action_stats, conn=conn)

#Creating/Updating the possession_stats database
pushToDB(table_name="possession_stats", df=possession_stats, conn=conn)

#Creating/Updating the playing_time_stats database
pushToDB(table_name="playing_time_stats", df=playing_time_stats, conn=conn)

#Creating/Updating the miscellaneous_stats database
pushToDB(table_name="miscellaneous_stats", df=miscellaneous_stats, conn=conn)


# Close the connection
conn.close()

print("Data Loading Phase Ended....")

Data Extract Phase Started....
Data Extract Phase Ended....
Data Transformation Phase Started....
Regular Season Transformations....
Standard Stats Transformations....
Goalkeeping Stats Transformations....
Advanced Goalkeeping Stats Transformations....
Shooting Stats Transformations....
Passing Stats Transformations....
Passing Types Stats Transformations....
Goal Shot Creation Stats Transformations....
Defensive Action Stats Transformations....
Posession Stats Transformations....
Playing Time Stats Transformations....
Miscellaneous Stats Transformations....
Data Transformation Phase Ended....
Establishing Connection with DB....
Successfully Established Connection with DB....
Data Loading Phase Started....
regular_season has been successfully committed.
standard_stats has been successfully committed.
goalkeeping_stats has been successfully committed.
advanced_goalkeeping_stats has been successfully committed.
shooting_stats has been successfully committed.
passing_stats has been succes

,Squad,#_Pl,90s,Tackles_Tkl,Tackles_TklW,Tackles_Def3rd,Tackles_Mid3rd,Tackles_Att3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Value
0,Arsenal,26,38.0,668,387,402,202,64,290,641,45.2,351,516,186,330,360,1028,841,18,opponent
1,Aston Villa,26,38.0,650,370,305,237,108,279,571,48.9,292,413,127,286,332,982,635,9,opponent
2,Bournemouth,31,38.0,711,426,318,271,122,344,665,51.7,321,404,96,308,279,990,563,13,opponent
3,Brentford,25,38.0,583,330,275,221,87,248,512,48.4,264,354,86,268,291,874,822,11,opponent
4,Brighton,29,38.0,603,350,312,220,71,279,622,44.9,343,505,160,345,417,1020,777,24,opponent
5,Chelsea,32,38.0,733,429,373,260,100,352,727,48.4,375,474,145,329,374,1107,792,10,opponent
6,Crystal Palace,26,38.0,771,436,370,299,102,314,681,46.1,367,449,136,313,366,1137,659,13,opponent
7,Everton,28,38.0,596,365,270,258,68,296,601,49.3,305,384,122,262,275,871,727,12,opponent
8,Fulham,29,38.0,617,362,280,258,79,238,507,46.9,269,447,116,331,340,957,754,8,opponent
9,Leeds United,29,38.0,749,427,379,292,78,328,596,55.0,268,509,140,369,391,1140,740,11,opponent
